In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/credit-card-approval-prediction/application_record.csv',header = 0)

In [ ]:
df_label= pd.read_csv('../input/credit-card-approval-prediction/credit_record.csv',header = 0)

In [ ]:
df_label

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.isnull().any()

In [ ]:
df.groupby('OCCUPATION_TYPE')['OCCUPATION_TYPE'].count()

In [ ]:
df_clean = df.dropna(axis=0)

In [ ]:
df_clean.isnull().any()

In [ ]:
df_clean_drop = df_clean.drop(['CODE_GENDER','DAYS_BIRTH', 'FLAG_EMAIL','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_MOBIL'],axis = 1)


In [ ]:
df_clean_drop.isnull().any()

In [ ]:
df_clean_drop.dtypes

In [ ]:
df_clean_drop['FLAG_OWN_CAR'] = pd.factorize(df_clean_drop['FLAG_OWN_CAR'])[0]
df_clean_drop['FLAG_OWN_REALTY'] = pd.factorize(df_clean_drop['FLAG_OWN_REALTY'])[0]
df_clean_drop['NAME_INCOME_TYPE'] = pd.factorize(df_clean_drop['NAME_INCOME_TYPE'])[0]
df_clean_drop['NAME_EDUCATION_TYPE'] = pd.factorize(df_clean_drop['NAME_EDUCATION_TYPE'])[0]
df_clean_drop['NAME_FAMILY_STATUS'] = pd.factorize(df_clean_drop['NAME_FAMILY_STATUS'])[0]
df_clean_drop['NAME_HOUSING_TYPE'] = pd.factorize(df_clean_drop['NAME_HOUSING_TYPE'])[0]
df_clean_drop['OCCUPATION_TYPE'] = pd.factorize(df_clean_drop['OCCUPATION_TYPE'])[0]

In [ ]:
df_clean_drop.dtypes

In [ ]:
X = df_clean_drop.values
X.shape

In [ ]:
df_label.groupby('ID')['STATUS'].count()

In [ ]:
df_label

In [ ]:
df_label['STATUS'], labels = pd.factorize(df_label['STATUS'])

In [ ]:
labels

In [ ]:
df_label

In [ ]:
df_label_clean = df_label[df_label['STATUS'] !=0]

In [ ]:
df_label_clean

In [ ]:
df_label_clean['STATUS'][df_label['STATUS'] > 2] = 1


In [ ]:
df_label_clean['STATUS'][df_label['STATUS'] <= 2] = 0


In [ ]:
df_label_clean.groupby('STATUS')['STATUS'].count()

In [ ]:
df_label_group = df_label_clean.groupby('ID').mean()



In [ ]:
df_label_group

> 0 คือลุกค้าดี แต่ 1 คือลุกค้าไม่ดีจ่ายหนี้นานกว่า 2 เดือนขึ้นไป

In [ ]:
df_label_group['STATUS'][df_label_group['STATUS'] > 0] = 1
df_label_group['STATUS'][df_label_group['STATUS'] <= 0] = 0

In [ ]:
df_label_group.groupby('STATUS')['STATUS'].count()


In [ ]:
df_label_group

In [ ]:
client_id = df_label_group.index.values
client_label = df_label_group['STATUS'].values

In [ ]:
df.describe()

In [ ]:
X_client_id = df_clean_drop['ID'].values
df_clean_drop = df_clean_drop.drop(['ID'],axis=1)

In [ ]:
X = df_clean_drop.values
X.shape

In [ ]:
real_client_id= [ item for item in X_client_id if item in client_id ]

In [ ]:
len(real_client_id)

In [ ]:
y = np.array([client_label[np.where(client_id == item)][0]for item in real_client_id])

In [ ]:
X_final = [X[np.where(X_client_id == item), :][0] for item in real_client_id]

In [ ]:
X_final[0].shape

In [ ]:
np.array(X_final).shape

In [ ]:
X_real_final = np.vstack(X_final)

In [ ]:
X_real_final.shape

In [ ]:

len(y)

In [ ]:
sum(y==0)

In [ ]:
sum(y==1)

In [ ]:
3089/(19648 + 3089)

# Model


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sm = RandomUnderSampler(random_state=0)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=0)



In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Logistic Regression


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm =  scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

params = {'C': [1, 2, 4, 8, 16, 32]}
clf = GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [ ]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['Good debt', 'Bad debt']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C,annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict,target_names=target_names))

# Deep learning

In [ ]:
import tensorflow as tf
def create_model():
    tf.random.set_seed(0)
    tf.compat.v1.reset_default_graph() # Clear Model
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(12, activation='relu', input_shape=(11,)),
      tf.keras.layers.Dense(6, activation='relu'),
      tf.keras.layers.Dense(4, activation='relu'),
      tf.keras.layers.Dense(2, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')  
    ])
    return model

In [ ]:
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 


In [ ]:
train_acc = list()
val_acc = list()
for i in range(0,140):
  history = model.fit(X_train_norm, y_train, epochs= 1, batch_size = 300, validation_data= (X_val_norm, y_val))
  tmp_avg = np.mean(history.history['accuracy'])
  tmp_avg_val = np.mean(history.history['val_accuracy'])
  train_acc.append(tmp_avg)
  val_acc.append(tmp_avg_val)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.figure(num=None, figsize=(16, 8), dpi=90, facecolor='w', edgecolor='k')
plt.plot()
plt.plot(train_acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
y_predict = np.round(model.predict(X_test_norm))
y_predict = [i[0] for i in y_predict.tolist()]
sum(y_predict == y_test)/len(y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
target_names = ['Good debt', 'Bad debt']
C = confusion_matrix(y_test,y_predict) 
C = C / C.astype(np.float).sum(axis=1)*100
sns.heatmap(C, annot=True, fmt=".2f",cmap="GnBu",xticklabels=target_names, yticklabels=target_names)
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.show()


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict,target_names=target_names))